In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
import os
import pandas as pd

# 📌 Universal BASE_DIR definition
try:
    # If running as a .py script
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # If running in Jupyter or IPython
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))


In [3]:
# # =========================
# # 1. LOADING WORLD COORDINATES
# # =========================

# 📄 Path to points in the world coordinate system
points_path = os.path.join(BASE_DIR, 'data', 'sensor', 'points', '2025-04-15_17-43-22_points_694020.csv')
df = pd.read_csv(points_path, skiprows=1, names=["X", "Y", "Z"])

# 🔍 Check
print(f"✅ Points loaded: {len(df)}")


✅ Points loaded: 694020


In [4]:
# Calculate Euclidean distances
deltas = np.sqrt(np.diff(df['X'])**2 + np.diff(df['Y'])**2 + np.diff(df['Z'])**2)
top_jumps = np.argpartition(deltas, -6)[-6:]  # Find indices of the 6 largest jumps
top_jumps = np.sort(top_jumps + 1)  # Shift by 1 to get split positions
scan_boundaries = [0] + top_jumps.tolist() + [len(df)]  # Include start and end
scans = [df.iloc[scan_boundaries[i]:scan_boundaries[i+1]].copy() for i in range(len(scan_boundaries) - 1)]

# Manually split scan 3 into two parts
scan3 = scans[2].reset_index(drop=True)
deltas_scan3 = np.sqrt(np.diff(scan3['X'])**2 + np.diff(scan3['Y'])**2 + np.diff(scan3['Z'])**2)
split_index = np.argmax(deltas_scan3) + 1  # Find largest jump
scans_cleaned = scans[:2] + [scan3.iloc[:split_index], scan3.iloc[split_index:]] + scans[3:6]

# Assign scan_id
df_labeled = df.copy()
df_labeled['scan_id'] = -1
start_idx = 0
for i, scan in enumerate(scans_cleaned):
    end_idx = start_idx + len(scan)
    df_labeled.loc[start_idx:end_idx-1, 'scan_id'] = i
    start_idx = end_idx


In [5]:
# =========================
# STEP: Z Outlier Removal (keep central 90% of values)
# =========================

z_low = df['Z'].quantile(0.05)
z_high = df['Z'].quantile(0.99)

print(f'🔍 Z range for filtering: {z_low:.2f} — {z_high:.2f}')

df_clean_z = df_labeled[(df_labeled['Z'] >= z_low) & (df_labeled['Z'] <= z_high)].copy()
print(f'✅ Points remaining after Z filtering: {len(df_clean_z)}')

# =========================
# ROI Filtering by X and Y (central region)
# =========================

x_min, x_max = df_clean_z['X'].quantile([0.3, 0.9])
y_min, y_max = df_clean_z['Y'].quantile([0.367, 0.495])
z_min, z_max = df_clean_z['Z'].quantile([0.6, 0.999])

print(f'🔍 ROI Z range: {z_min:.2f} — {z_max:.2f}')
print(f'🔍 ROI X range: {x_min:.2f} — {x_max:.2f}')
print(f'🔍 ROI Y range: {y_min:.2f} — {y_max:.2f}')

df_clean_roi = df_clean_z[
    (df_clean_z['X'] >= x_min) & (df_clean_z['X'] <= x_max) &
    (df_clean_z['Y'] >= y_min) & (df_clean_z['Y'] <= y_max) &
    (df_clean_z['Z'] >= z_min) & (df_clean_z['Z'] <= z_max)
].copy()

print(f'✅ Points remaining after ROI filtering: {len(df_clean_roi)}')

# ✅ Rename column
df_clean_roi = df_clean_roi.rename(columns={
    "scan_id": "layer_id_5MethodGeometryBased"
})


🔍 Z range for filtering: -42.35 — -31.16
✅ Points remaining after Z filtering: 652386
🔍 ROI Z range: -41.67 — -31.21
🔍 ROI X range: 1819.05 — 1874.29
🔍 ROI Y range: 348.98 — 356.71
✅ Points remaining after ROI filtering: 22965


In [6]:
import plotly.graph_objs as go

# Unique global layers
unique_layers = sorted(df_clean_roi['layer_id_5MethodGeometryBased'].unique())  # global_layer_id

# 🎯 Points for each layer
data = []
for lid in unique_layers:
    subset = df_clean_roi[df_clean_roi['layer_id_5MethodGeometryBased'] == lid]
    trace = go.Scatter3d(
        x=subset['X'],
        y=subset['Y'],
        z=subset['Z'],
        mode='markers',
        marker=dict(size=2),
        name=f'Layer {lid}',
        visible=True if lid == 0 else False  # Show only the first layer by default
    )
    data.append(trace)

# 🔘 Buttons to switch between layers
buttons = []
for i, lid in enumerate(unique_layers):
    visible = [j == i for j in range(len(unique_layers))]
    buttons.append(dict(
        label=f"Layer {lid}",
        method="update",
        args=[{"visible": visible}, {"title": f"3D: Only Layer {lid}"}]
    ))

# Button to show all layers
buttons.insert(0, dict(
    label="All Layers",
    method="update",
    args=[{"visible": [True]*len(unique_layers)}, {"title": "3D: All Layers"}]
))

# ⚙️ Final chart assembly
fig = go.Figure(data=data)
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        showactive=True,
        x=1.05, y=1,
        buttons=buttons
    )],
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title="3D: Only Layer 0 (default)"
)

# # 👁 Show
# fig.show()
pass


In [7]:
import os

# 📁 Path to the output directory next to this notebook
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# 📄 Path to the HTML file
output_path = os.path.join(output_dir, "3D_GeometryBased_layers.html")

# 💾 Save the figure
fig.write_html(output_path)
print(f"✅ Plot saved: {output_path}")


✅ Plot saved: ./output\3D_GeometryBased_layers.html


In [8]:
# ✅ Enhanced visualization with color per layer and combined 3D view (Geometry-Based Method)

import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import numpy as np

# 📁 Output folder
output_dir = "./output/plots_GeometryBased_layers"
os.makedirs(output_dir, exist_ok=True)

# 📌 Unique layers and colormap
unique_layers = sorted(df_clean_roi['layer_id_5MethodGeometryBased'].unique())
n_layers = len(unique_layers)
colormap = cm.get_cmap('tab20', n_layers)
layer_colors_geometry_based = {layer_id: colormap(i) for i, layer_id in enumerate(unique_layers)}

# 📐 Coordinate bounds
x_min, x_max = df_clean_roi['X'].min(), df_clean_roi['X'].max()
y_min, y_max = df_clean_roi['Y'].min(), df_clean_roi['Y'].max()
z_min, z_max = df_clean_roi['Z'].min(), df_clean_roi['Z'].max()
aspect = [x_max - x_min, y_max - y_min, z_max - z_min]

# 📊 2D and 3D plots for each layer
for gid in unique_layers:
    layer = df_clean_roi[df_clean_roi['layer_id_5MethodGeometryBased'] == gid]
    print(f'🟢 Visualizing layer {gid} ({len(layer)} points)')

    # 2D projections
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    axs[0].scatter(layer['X'], layer['Z'], s=1, color=layer_colors_geometry_based[gid])
    axs[0].set_title(f'Layer {gid} — XZ')
    axs[0].set_xlabel('X')
    axs[0].set_ylabel('Z')
    axs[0].set_xlim(x_min, x_max)
    axs[0].set_ylim(z_min, z_max)
    axs[0].grid(True)

    axs[1].scatter(layer['Y'], layer['Z'], s=1, color=layer_colors_geometry_based[gid])
    axs[1].set_title(f'Layer {gid} — YZ')
    axs[1].set_xlabel('Y')
    axs[1].set_ylabel('Z')
    axs[1].set_xlim(y_min, y_max)
    axs[1].set_ylim(z_min, z_max)
    axs[1].grid(True)

    axs[2].scatter(layer['X'], layer['Y'], s=1, color=layer_colors_geometry_based[gid])
    axs[2].set_title(f'Layer {gid} — XY')
    axs[2].set_xlabel('X')
    axs[2].set_ylabel('Y')
    axs[2].set_xlim(x_min, x_max)
    axs[2].set_ylim(y_min, y_max)
    axs[2].grid(True)

    plt.tight_layout()
    plot_2d_path = os.path.join(output_dir, f"layer_{gid}_2D.png")
    fig.savefig(plot_2d_path, dpi=300)
    plt.close(fig)

    # 3D plot for a single layer
    fig3d = plt.figure(figsize=(8, 6))
    ax = fig3d.add_subplot(111, projection='3d')
    ax.scatter(layer['X'], layer['Y'], layer['Z'], s=1, color=layer_colors_geometry_based[gid])
    ax.set_title(f'Layer {gid} — 3D')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)
    ax.set_box_aspect(aspect)
    ax.view_init(elev=30, azim=225)

    plot_3d_path = os.path.join(output_dir, f"layer_{gid}_3D.png")
    fig3d.savefig(plot_3d_path, dpi=300)
    plt.close(fig3d)

    print(f"💾 Saved: {plot_2d_path}, {plot_3d_path}")

# 🧩 Combined 3D plot of all layers
fig_all = plt.figure(figsize=(10, 8))
ax_all = fig_all.add_subplot(111, projection='3d')

for gid in unique_layers:
    layer = df_clean_roi[df_clean_roi['layer_id_5MethodGeometryBased'] == gid]
    ax_all.scatter(layer['X'], layer['Y'], layer['Z'], s=1, color=layer_colors_geometry_based[gid], label=f'Layer {gid}')

ax_all.set_title('3D Layered Point Cloud View (Geometry-Based Method)')
ax_all.set_xlabel('X')
ax_all.set_ylabel('Y')
ax_all.set_zlabel('Z')
ax_all.set_xlim(x_min, x_max)
ax_all.set_ylim(y_min, y_max)
ax_all.set_zlim(z_min, z_max)
ax_all.set_box_aspect(aspect)
ax_all.view_init(elev=30, azim=225)

plot_all_path = os.path.join(output_dir, "all_layers_3D.png")
fig_all.savefig(plot_all_path, dpi=300)
plt.close(fig_all)

print(f"📌 Combined 3D plot saved: {plot_all_path}")


🟢 Visualizing layer 1 (3316 points)


C:\Users\kopil\AppData\Local\Temp\ipykernel_4348\2125352885.py:16: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



💾 Saved: ./output/plots_GeometryBased_layers\layer_1_2D.png, ./output/plots_GeometryBased_layers\layer_1_3D.png
🟢 Visualizing layer 2 (3704 points)
💾 Saved: ./output/plots_GeometryBased_layers\layer_2_2D.png, ./output/plots_GeometryBased_layers\layer_2_3D.png
🟢 Visualizing layer 3 (3873 points)
💾 Saved: ./output/plots_GeometryBased_layers\layer_3_2D.png, ./output/plots_GeometryBased_layers\layer_3_3D.png
🟢 Visualizing layer 4 (3936 points)
💾 Saved: ./output/plots_GeometryBased_layers\layer_4_2D.png, ./output/plots_GeometryBased_layers\layer_4_3D.png
🟢 Visualizing layer 5 (4041 points)
💾 Saved: ./output/plots_GeometryBased_layers\layer_5_2D.png, ./output/plots_GeometryBased_layers\layer_5_3D.png
🟢 Visualizing layer 6 (4095 points)
💾 Saved: ./output/plots_GeometryBased_layers\layer_6_2D.png, ./output/plots_GeometryBased_layers\layer_6_3D.png
📌 Combined 3D plot saved: ./output/plots_GeometryBased_layers\all_layers_3D.png


In [9]:
# 💾 Save ROI-filtered data
output_dir = os.path.join(BASE_DIR, "src", "5MethodGeometryBased", "output")
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "5MethodGeometryBased.csv")
df_clean_roi.to_csv(output_path, index=False)

print(f"✅ Saved: {output_path}")


✅ Saved: c:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\5MethodGeometryBased\output\5MethodGeometryBased.csv
